<a href="https://colab.research.google.com/github/heidyfp/Aula4/blob/main/IA_Google%2B_Alura_Aula_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U -q google-generativeai

In [2]:
import numpy as np
import pandas as pd
import google.generativeai as genai
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('SECRET_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [5]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


In [14]:
#Ejemplo de embedding
title = 'A próxima generação de IA para desenvolvedores e Google Workspace'
sample_text = ( 'Titulo:A próxima generação de IA para desenvolvedores e Google Workspace'
'\n'
'Artigo Completo'
'\n'
'Gemini API & Google IA Studio: Una materia accesible de explorar y crear prototipos con aplicaciones de IA generativas'

)

embedding = genai.embed_content(model="models/embedding-001",
                                      content=sample_text,
                                      title=title,
                                      task_type = "RETRIEVAL_DOCUMENT")
print(embedding)


{'embedding': [0.05409523, -0.029174177, -0.0307718, 0.011006993, 0.08204209, 0.0032902933, -0.03555953, -0.021569764, 0.06421903, 0.06790191, 0.018536553, 0.030903606, -0.044005502, -0.020199787, 0.0070433086, -0.009895312, 0.0290733, -0.017249966, -0.03267027, 0.0010824124, 0.009649257, 0.011283337, -0.040399417, -0.069273554, -0.029203499, 0.022766273, 0.013130985, -0.048043832, -0.029252555, 0.028755428, -0.056379724, 0.046211246, -0.042203695, 0.015026432, -0.044828042, -0.035443407, -0.027252192, -0.052856453, 0.005837106, 0.012332278, -0.008428515, -0.08136561, -0.016149946, 0.023108726, 0.010695362, -0.025697121, 0.04554527, 0.03585742, 0.025925957, -0.05355938, 0.011843076, 0.016169172, 0.05647707, -0.038550887, -0.006705653, -0.007331963, 0.020557314, -0.043055464, 0.022383967, -0.004630676, 0.0064912303, 0.02101791, -0.00857838, 0.05231312, 0.022617454, -0.05975579, -0.046821713, -0.0016861687, 0.014644702, 0.042973917, 0.0022005413, 0.011575038, 0.0529175, -0.0635702, -0.04

In [15]:
#Listando Documentos que seran buscados
DOCUMENT1 = {
    "title": "Operating the Climate Control System",
    "content": "Your Googlecar has a climate control system that allows you to adjust the temperature and airflow in the car. To operate the climate control system, use the buttons and knobs located on the center console.  Temperature: The temperature knob controls the temperature inside the car. Turn the knob clockwise to increase the temperature or counterclockwise to decrease the temperature. Airflow: The airflow knob controls the amount of airflow inside the car. Turn the knob clockwise to increase the airflow or counterclockwise to decrease the airflow. Fan speed: The fan speed knob controls the speed of the fan. Turn the knob clockwise to increase the fan speed or counterclockwise to decrease the fan speed. Mode: The mode button allows you to select the desired mode. The available modes are: Auto: The car will automatically adjust the temperature and airflow to maintain a comfortable level. Cool: The car will blow cool air into the car. Heat: The car will blow warm air into the car. Defrost: The car will blow warm air onto the windshield to defrost it."}

DOCUMENT2 = {
    "title": "Touchscreen",
    "content": "Your Googlecar has a large touchscreen display that provides access to a variety of features, including navigation, entertainment, and climate control. To use the touchscreen display, simply touch the desired icon.  For example, you can touch the \"Navigation\" icon to get directions to your destination or touch the \"Music\" icon to play your favorite songs."}

DOCUMENT3 = {
    "title": "Shifting Gears",
    "content": "Your Googlecar has an automatic transmission. To shift gears, simply move the shift lever to the desired position.  Park: This position is used when you are parked. The wheels are locked and the car cannot move. Reverse: This position is used to back up. Neutral: This position is used when you are stopped at a light or in traffic. The car is not in gear and will not move unless you press the gas pedal. Drive: This position is used to drive forward. Low: This position is used for driving in snow or other slippery conditions."}

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

In [16]:
df = pd.DataFrame(documents)
df.columns = ["Título","Conteudo"]
df

,Título,Conteudo
0,Operating the Climate Control System,Your Googlecar has a climate control system th...
1,Touchscreen,Your Googlecar has a large touchscreen display...
2,Shifting Gears,Your Googlecar has an automatic transmission. ...


In [17]:
model = "models/embedding-001"
def embed_fn(title,text, model):
  return genai.embed_content(model=model,
                                      content=text,
                                      title=title,
                                      task_type = "RETRIEVAL_DOCUMENT")['embedding']


In [18]:
df["Embeddings"] = df.apply(lambda row:embed_fn(row["Título"], row["Conteudo"]),axis=1)
df

,Título,Conteudo,Embeddings
0,Operating the Climate Control System,Your Googlecar has a climate control system th...,"[-0.033361107, -0.021217084, -0.049581926, -0...."
1,Touchscreen,Your Googlecar has a large touchscreen display...,"[0.009660736, -0.030662702, -0.017281422, -0.0..."
2,Shifting Gears,Your Googlecar has an automatic transmission. ...,"[-0.04270796, -0.007160868, -0.03242516, -0.02..."


In [19]:
def gerar_e_buscar_consulta(consulta, base, model):
  embedding_da_consulta = genai.embed_content(model=model,
                                      content=consulta,
                                      task_type = "RETRIEVAL_QUERY")
  productos_escalares = np.dot(np.stack(df['Embeddings']), embedding_da_consulta["embedding"])
  idice = np.argmax(productos_escalares)
  return df.iloc[idice]['Conteudo'] # Return text from index with max value

In [20]:
consulta = 'Como faço para trocar marchas en un carro do Google'
trecho = gerar_e_buscar_consulta(consulta, df, model)
trecho

'Your Googlecar has a large touchscreen display that provides access to a variety of features, including navigation, entertainment, and climate control. To use the touchscreen display, simply touch the desired icon.  For example, you can touch the "Navigation" icon to get directions to your destination or touch the "Music" icon to play your favorite songs.'

In [23]:
generation_config = {
    "candidate_count": 1,
    "temperature": 0,
}


In [24]:
prompt = f'Reescreva esse texto de uma forma mais descontraída, sem adicionar informações que no façam  parte do texto: {trecho}'
model_2 = genai.GenerativeModel('gemini-1.0-pro', generation_config=generation_config)
response = model_2.generate_content(prompt)
print(response.text)



Seu Googlecar tem uma telona sensível ao toque que te dá acesso a um monte de coisas legais, tipo GPS, música e até pra controlar o ar-condicionado. Pra usar a telona, é só tocar no ícone que você quer. Por exemplo, toca no "GPS" pra saber como chegar no seu destino ou no "Música" pra tocar suas músicas preferidas.
